<a href="https://colab.research.google.com/github/muhamedezzat/Pyhthon-Applications/blob/master/Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Disney Dataset Creation with Python & BeautifulSoup4**

Scrape &clean a list of all Disney films from wikipedia pages  to create a dataset for further analysis.

# **Task #1 Get info box data and store it in python dictionary**

1) Import Nesecary libraries: 

**requests** >>>>>> get url link and make request to web page 

**BeautifulSoup4** >>>>>>>>> Scraping with html pages 




In [ ]:
import requests 
from bs4 import BeautifulSoup as bs

2) Load web pages Using requests library 

In [ ]:
r = requests.get("https://en.wikipedia.org/wiki/That_Darn_Cat_(1997_film)")

3) Convert web page to BeautifulSoup Object 

In [ ]:
soup = bs(r.content)
def clean_tags(soup):
  for tag in soup.find_all(["sup","span"]):
    tag.decompose() 



4) Grapping HTML Objects to deal with its to extract a data 

In [ ]:
# we use find method to grap HTML object by Selectors 
# find()：Gets the first tag of the incoming HTML object that satisfies the condition and returns. A function of a label group or a single label.
# findAll()：Get all the conditions of the incoming HTML object and return it. Prototype: findAll(tag, atributes, recursive, text, limit, keywords)

infoBox = soup.find(class_="infobox vevent")
infoRows = infoBox.find_all("tr")
clean_tags(soup)

Now I can access to all data in box info 

Now I need to extract data in form filmee name (title) and their data and stored it in python dictionary 

`I use Python enumerate() to  adds counter to an iterable and returns it (the enumerate object).`

In [ ]:
movieInfo = { }

# get_Conteent_Value to loop in any itam have a list of names and out it in form list not text 

def get_Content_Value(rowData):
  if rowData.find("li"):
    return [li.get_text(" ",strip=True).replace("\xa0"," ")  for li in rowData.find_all("li")] 
    
  elif rowData.find("br"):
        return [text for text in rowData.stripped_strings]
  else:
    return rowData.get_text(" ",strip=True).replace("\xa0"," ")

for index , row in enumerate(infoRows):
  if index == 0 :
    movieInfo["title"] = row.find("th").get_text(" ",strip=True)
  elif index == 1:
    continue
  else:
    contentKey= row.find("th").get_text(" ",strip=True)
    contentValue= get_Content_Value(row.find("td"))
    movieInfo[contentKey] = contentValue

for key , value in movieInfo.items():
  print(f"{key} : {value}")


# **Task #2 Get info box for all films**


In [ ]:
import requests 
from bs4 import BeautifulSoup as bs


def get_Content_Value(rowData):
    if rowData.find("li"):
      return [li.get_text(" ",strip=True).replace("\xa0"," ")  for li in rowData.find_all("li")] 

    elif rowData.find("br"):
        return [text for text in rowData.stripped_strings]
    
    else:
      return rowData.get_text(" ",strip=True).replace("\xa0"," ")

def clean_tags(soup):
  for tag in soup.find_all(["sup","span"]):
    tag.decompose()

def get_All_Films(fullPath):
  r = requests.get(fullPath)
  soup = bs(r.content)

  infoBox = soup.find(class_="infobox vevent")
  infoRows = infoBox.find_all("tr")
  clean_tags(soup)
  
  movieInfo = {}

  for index , row in enumerate(infoRows):
    if index == 0 :
      movieInfo["title"] = row.find("th").get_text(" ",strip=True)
    
    else:
      header = row.find('th')
      if header:
        contentKey= row.find("th").get_text(" ",strip=True)
        contentValue= get_Content_Value(row.find("td"))
        movieInfo[contentKey] = contentValue
  
  return movieInfo


In [ ]:
allreq = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
soup2 = bs(allreq.content)
filmTables = soup2.select(".wikitable.sortable i a")

basePath = "https://en.wikipedia.org/"

movieList = [ ]
for index , movie in enumerate(filmTables):
  # if index == 2:
  #   break
  # if index % 10 == 0:
  #   print(index)

  try:
    relativePath = movie["href"]
    fullPath = basePath + relativePath
    title = movie["title"]
    movieList.append(get_All_Films(fullPath))
  
  except Exception as e:
      print(movie.get_text())
      print(e)

# print(movieList)
print(len(movieList()))

# **Now we are going to save data**

In [72]:
import json
def save(title,data):
  with open(title,"w",encoding="utf-8") as f:
    json.dump(data,f,ensure_ascii=False,indent=3)

In [73]:
def load(title):
  with open(title,encoding="utf-8") as f:
    return json.load(f)

In [ ]:
save("Disney Data clean.json",movieList)

# **Task #3 Clean Our Data**

In [6]:
movieList = load("Disney Data clean.json")

## **Sub-Tasks:** 

1) Clean up references [1]

2) Convert running time into intger

3) Conver datas int datetime object

4) split up long string 

5) Convert budget & box office into numerical values 

In [7]:
# Clean up references [1]
# we go to clean it from html by its specific tag with decompose() for deleted it and extract()

def clean_tags(soup):
  for tag in soup.find_all(["sup","span"]):
    tag.decompose()

# we put this function after creating soup object fron html page to temove this tage from html page before extract data 
# sup tag form reference
#span to remove data like this (1999-10-12)

In [ ]:
# split up long string 


# i am use thtis block of code on data get from html page to check if "br" found 
# between string to stripped it and return the blocks of string in list format 
# in this code we use list comprehension
# We can now identify where list comprehensions are used.
# the data return is a string, not a list. This is the power of list comprehension. 
# It can identify when it receives a string or a tuple and work on it like a list.
#  syntax [expression for item in list]
# expersion same the item 

elif rowData.find("br"):
        return [text for text in rowData.stripped_strings]

In [ ]:
# Convert running time into intger
#access to all moive running time and because it is a string we use list comprehension 
print([movie.get("Running time", "N/A") for movie in movieList])

In [ ]:
def minutes_To_Int(runningTime):
  if runningTime == "N/A":   # running time  N/A
    return None
  if isinstance(runningTime,list):  # running time  LIST
    return int(runningTime[0].split(" ")[0])
  else:                                   ## running time  STRING        
    return int(runningTime.split(" ")[0])

for movie in movieList:
  movie["Running time"] = minutes_To_Int(movie.get("Running time","N/A"))


In [ ]:
# Convert budget & box office into numerical values
# as Running time part 

[movie.get("Budget" ,"N/A") for movie in movieList]

In [14]:
import re

amounts = r"thousand|million|billion"
number = r"\d+(,\d{3})*\.*\d*"

word_re = rf"\${number}(-|\sto\s|–)?({number})?\s({amounts})"
value_re = rf"\${number}"

def word_to_value(word):
    value_dict = {"thousand": 1000, "million": 1000000, "billion": 1000000000}
    return value_dict[word]

def parse_word_syntax(string):
    value_string = re.search(number, string).group()
    value = float(value_string.replace(",", ""))
    word = re.search(amounts, string, flags=re.I).group().lower()
    word_value = word_to_value(word)
    return value*word_value


def parse_value_syntax(string):
    value_string = re.search(number, string).group()
    value = float(value_string.replace(",", ""))
    return value

def money_conversion(money):
    if money == "N/A":
        return None

    if isinstance(money, list):
        money = money[0]

    word_syntax = re.search(word_re, money, flags=re.I)
    value_syntax = re.search(value_re, money)

    if word_syntax:
        return parse_word_syntax(word_syntax.group())

    elif value_syntax:
        return parse_value_syntax(value_syntax.group())
    
    else:
      return None

In [15]:
for movie in movieList:
  movie["Budget (float)"] = money_conversion(movie.get("Budget","N/A"))
  movie["Box office (float)"] = money_conversion(movie.get("Box office","N/A"))

In [ ]:
# Convert datas int datetime object
print([movie.get("Release date","N\A") for movie in movieList])

In [69]:
# june 26, 1950
from datetime import datetime

dates = [movie.get("Release date","N/A") for movie in movieList]
def clean_date(date):
  return date.split("(")[0].strip()

def date_conversion(date):
    if isinstance(date, list):
        date = date[0]
    if date == "N/A":
      return None
        
    dateStr = clean_date(date)
    fmts = ["%B %d, %Y", "%d %B %Y"]
    for fmt in fmts:
        try:
            return datetime.strptime(dateStr, fmt)
        except:
            pass
    return None

In [67]:
for movie in movieList:
  movie["Release date (datetime)"] = date_conversion(movie.get("Release date","N/A"))

In [80]:
import pickle

def save_data_pickle(name, data):
    with open(name, 'wb') as f:
        pickle.dump(data, f)

In [81]:
def load_data_pickle(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [82]:
save_data_pickle("disney_movie_data_cleaned_more.pickle", movieList)

In [ ]:
a = load_data_pickle("disney_movie_data_cleaned_more.pickle")

In [ ]:
a == movieList

# **Task #4 Attach IMDB /Rotten tomato Scores**

In [ ]:
# http://www.omdbapi.com/?apikey=[yourkey]&

In [ ]:
import requests
import urllib
import os
def get_omdb_info(title):
  api_key = "af390a9"
  base_url = "http://www.omdbapi.com/?"
  parameters = {"apikey" : "af390a9" , 't': title}
  params_encoded = urllib.parse.urlencode(parameters)
  full_url = base_url + params_encoded
  return requests.get(full_url).json()

def get_rotten_tomato_score(omdb_info):
  ratings = omdb_info.get('Ratings', [])
  for rating in ratings:
    if rating['Source'] == 'Rotten Tomatoes':
      return rating['Value']
  return None

get_omdb_info("into the woods")

In [ ]:
for movie in movieList:
    title = movie['title']
    omdb_info = get_omdb_info(title)
    movie['imdb'] = omdb_info.get('imdbRating', None)
    movie['metascore'] = omdb_info.get('Metascore', None)
    movie['rotten_tomatoes'] = get_rotten_tomato_score(omdb_info)

# **Task #6 Save data in CSV and json file** 

In [ ]:
movieList

In [111]:
movieListCopy = [movie.copy() for movie in movieList]

In [112]:
for movie in movieListCopy:
    currentDate = movie['Release date (datetime)']
    if currentDate:
        movie['Release date (datetime)'] = currentDate.strftime("%B %d, %Y")
    else:
        movie['Release date (datetime)'] = None
    

In [114]:
save("disney_data_final.json", movieListCopy)

In [117]:
#Convert data to CSV

import pandas as pd 
df = pd.DataFrame(movieList)

In [118]:
df.to_csv("disney_movie_data_final.csv")

In [ ]:
df.info()

In [ ]:
boxOffice = df.sort_values(['Box office (float)'],  ascending=False)
boxOffice.head()